
# Amazon Sales Analysis — Internship Project

This notebook performs a **clear, step-by-step analysis** of the Amazon sales dataset, aligned with the project objectives:
1) Sales Overview  
2) Product Analysis  
3) Fulfilment Analysis  
4) Customer Segmentation  
5) Geographical Analysis  
6) Business Insights & Recommendations  

**How to use:**
- Put the CSV at the path you set in `csv_path` below. By default, it looks for `./data/Amazon Sale Report.csv`.
- Run cells **top to bottom**. Every step is explained and uses readable code (no dense one-liners).
- Charts are generated with matplotlib and appear inline.


## 1) Imports

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Chart display settings (no custom colors/styles to keep it simple)
%matplotlib inline



## 2) Load the dataset (with robust encoding)
- We try common encodings to avoid errors.
- Adjust `csv_path` if your file is elsewhere.


In [ ]:

# Set your CSV path here. Suggested structure is ./data/Amazon Sale Report.csv
csv_path = "./data/Amazon Sale Report.csv"

# If you're running this in Colab:
# 1) Upload the CSV using the file pane OR
# 2) Mount Drive and point csv_path to the file on Drive.

encodings_to_try = ["utf-8", "latin1", "cp1252"]
last_err = None
df = None
for enc in encodings_to_try:
    try:
        df = pd.read_csv(csv_path, low_memory=False, encoding=enc)
        print(f"Loaded CSV with encoding: {enc}")
        break
    except Exception as e:
        last_err = e

if df is None:
    # Fallback for this environment if the path above doesn't exist.
    # You can remove this block after setting a proper csv_path.
    try:
        df = pd.read_csv("/mnt/data/Amazon Sale Report.csv", low_memory=False, encoding="latin1")
        print("Loaded CSV from /mnt/data/ with latin1 encoding (fallback).")
    except Exception as e:
        raise RuntimeError(f"Failed to load CSV. Last error: {last_err}") from e

print("Rows:", len(df))
df.head(3)


## 3) Clean and standardize columns
- Convert column names to lowercase with underscores.
- Detect important columns by common names.
- Parse dates; coerce amounts/qty to numeric.


In [ ]:

# Standardize column names
df.columns = [c.strip().lower().replace(" ", "_").replace("-", "_") for c in df.columns]

# Helper to find columns by candidate names
def find_col(candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_order_id   = find_col(["order_id", "order_id_"])
col_date       = find_col(["date", "order_date", "purchase_date"])
col_status     = find_col(["status"])
col_fulfilment = find_col(["fulfilment", "fulfillment", "fulfillment_by"])
col_category   = find_col(["category", "product_category"])
col_size       = find_col(["size"])
col_qty        = find_col(["qty", "quantity"])
col_amount     = find_col(["amount", "total", "item_total", "grand_total"])
col_ship_city  = find_col(["ship_city", "city"])
col_ship_state = find_col(["ship_state", "state"])
col_b2b        = find_col(["b2b"])

# Parse date and numerics safely
if col_date:
    df[col_date] = pd.to_datetime(df[col_date], errors="coerce")

if col_amount is None:
    df["__amount__"] = np.nan
    col_amount = "__amount__"
df[col_amount] = pd.to_numeric(df[col_amount], errors="coerce")

if col_qty is None:
    df["__qty__"] = 1.0
    col_qty = "__qty__"
df[col_qty] = pd.to_numeric(df[col_qty], errors="coerce")

# Quick info
summary_cols = [col for col in [col_order_id, col_date, col_status, col_fulfilment, col_category, col_size, col_qty, col_amount, col_ship_city, col_ship_state, col_b2b] if col]
print("Detected columns:", summary_cols)
df[summary_cols].head(5) if summary_cols else df.head(5)


## 4) Core KPIs
We calculate:
- Total Orders
- Total Sales (sum of `amount`)
- Average Order Value (AOV)
- Date range


In [ ]:

total_orders = len(df)
total_sales = df[col_amount].sum(skipna=True)
avg_order_value = total_sales / total_orders if total_orders else np.nan
date_min = df[col_date].min() if col_date else None
date_max = df[col_date].max() if col_date else None

print(f"""KPIs
-----
Total Orders: {total_orders:,}
Total Sales: {total_sales:,.2f}
Average Order Value (AOV): {avg_order_value:,.2f}
Date Range: {date_min.date() if pd.notna(date_min) else 'N/A'} to {date_max.date() if pd.notna(date_max) else 'N/A'}
""")


## 5) Sales Overview — Trends Over Time
- We group revenue by day and plot the time series.


In [ ]:

if col_date:
    sales_over_time = (
        df.dropna(subset=[col_date])
          .groupby(df[col_date].dt.date)[col_amount].sum()
          .reset_index(name="sales")
          .rename(columns={col_date:"date"})
          .sort_values("date")
    )
    display(sales_over_time.head(10))
    
    # Plot
    plt.figure(figsize=(10,5))
    plt.plot(sales_over_time["date"], sales_over_time["sales"])
    plt.title("Sales Over Time (Daily)")
    plt.xlabel("Date")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("No date column detected; skipping sales-over-time analysis.")


## 6) Product Analysis — Categories and Sizes
- Top categories by **revenue** and **units**.
- Sizes by **units**.


In [ ]:

if col_category:
    by_category = (
        df.groupby(col_category).agg(
            orders=(col_order_id if col_order_id else col_amount, "count"),
            units=(col_qty, "sum"),
            revenue=(col_amount, "sum")
        ).reset_index().sort_values("revenue", ascending=False)
    )
    display(by_category.head(10))
    
    # Plot top 10 categories by revenue
    top = by_category.head(10)
    plt.figure(figsize=(10,5))
    plt.bar(top[col_category], top["revenue"])
    plt.title("Top Categories by Revenue")
    plt.xlabel("Category")
    plt.ylabel("Revenue")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("No category column detected; skipping category analysis.")

if col_size:
    by_size = (
        df.groupby(col_size).agg(
            orders=(col_order_id if col_order_id else col_amount, "count"),
            units=(col_qty, "sum"),
            revenue=(col_amount, "sum")
        ).reset_index().sort_values("units", ascending=False)
    )
    display(by_size.head(10))
    
    # Plot top sizes by units
    top = by_size.head(10)
    plt.figure(figsize=(10,5))
    plt.bar(top[col_size], top["units"])
    plt.title("Sizes by Units Sold")
    plt.xlabel("Size")
    plt.ylabel("Units")
    plt.tight_layout()
    plt.show()
else:
    print("No size column detected; skipping size analysis.")


## 7) Fulfilment Analysis — Method × Status
- Compare order counts by fulfilment method and status.


In [ ]:

if col_fulfilment and col_status:
    fulfilment_status = (
        df.groupby([col_fulfilment, col_status]).size().reset_index(name="orders")
        .sort_values("orders", ascending=False)
    )
    display(fulfilment_status.head(10))
    
    # Simple top-10 combo plot
    top_fs = fulfilment_status.head(10)
    labels = [f"{a} | {b}" for a,b in zip(top_fs[col_fulfilment], top_fs[col_status])]
    plt.figure(figsize=(10,5))
    plt.bar(labels, top_fs["orders"])
    plt.title("Orders by Fulfilment & Status (Top 10)")
    plt.xlabel("Fulfilment | Status")
    plt.ylabel("Orders")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("Missing fulfilment or status column; skipping fulfilment analysis.")


## 8) Geographical Analysis — States and Cities
- Top states and cities by orders and revenue.


In [ ]:

if col_ship_state:
    top_states = (
        df.groupby(col_ship_state).agg(
            orders=(col_order_id if col_order_id else col_amount, "count"),
            revenue=(col_amount, "sum")
        ).reset_index().sort_values("orders", ascending=False)
    )
    display(top_states.head(10))
    
    # Plot top 10 states
    ts = top_states.head(10)
    plt.figure(figsize=(10,5))
    plt.bar(ts[col_ship_state], ts["orders"])
    plt.title("Top States by Orders")
    plt.xlabel("State")
    plt.ylabel("Orders")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
else:
    print("No ship_state column detected; skipping state analysis.")

if col_ship_city:
    top_cities = (
        df.groupby(col_ship_city).agg(
            orders=(col_order_id if col_order_id else col_amount, "count"),
            revenue=(col_amount, "sum")
        ).reset_index().sort_values("orders", ascending=False)
    )
    display(top_cities.head(10))
else:
    print("No ship_city column detected; skipping city-level listing.")


## 9) Customer Segmentation (Simple Proxy)
- If a `b2b` flag exists, use B2B vs B2C.
- Otherwise, segment orders by order value tertiles: **Low**, **Mid**, **High**.


In [ ]:

if col_b2b and df[col_b2b].notna().any():
    seg_table = df.groupby(col_b2b).agg(
        orders=(col_order_id if col_order_id else col_amount, "count"),
        revenue=(col_amount, "sum"),
        aov=(col_amount, "mean")
    ).reset_index()
    display(seg_table)
else:
    # Proxy segmentation by order value
    value = df[col_amount].fillna(0)
    cuts = pd.qcut(value.rank(method="first"), q=3, labels=["Low-value", "Mid-value", "High-value"])
    df["value_segment"] = cuts
    seg_table = df.groupby("value_segment").agg(
        orders=(col_order_id if col_order_id else col_amount, "count"),
        revenue=(col_amount, "sum"),
        aov=(col_amount, "mean")
    ).reset_index().sort_values("aov")
    display(seg_table)
    
    # Pie of orders share by segment
    plt.figure(figsize=(6,6))
    plt.pie(seg_table["orders"], labels=list(seg_table["value_segment"].astype(str)), autopct="%1.1f%%")
    plt.title("Customer Segments (Proxy)")
    plt.tight_layout()
    plt.show()


## 10) Business Insights & Recommendations
- **Inventory:** Keep fast-moving categories/sizes stocked; set reorder points and safety stock.  
- **Fulfilment:** Track on-time delivery; standardize merchant fulfilment quality to reduce returns/cancels.  
- **Targeting:** Tailor campaigns by segment and region to improve conversion.  
- **Forecasting:** Use trends for demand forecasting and capacity planning.  

> Replace or add specific actions after reviewing your actual top categories, sizes, states, and fulfilment outcomes above.



## (Optional) 11) Export key tables for your report/README


In [ ]:

# Create an `exports` folder to save transparent CSVs if you want to include them in your repo
import os
os.makedirs("./exports", exist_ok=True)

if 'by_category' in globals():
    by_category.to_csv("./exports/by_category.csv", index=False)
if 'by_size' in globals():
    by_size.to_csv("./exports/by_size.csv", index=False)
if 'fulfilment_status' in globals():
    fulfilment_status.to_csv("./exports/fulfilment_status.csv", index=False)
if 'top_states' in globals():
    top_states.to_csv("./exports/top_states.csv", index=False)
if 'top_cities' in globals():
    top_cities.to_csv("./exports/top_cities.csv", index=False)
if 'sales_over_time' in globals():
    sales_over_time.to_csv("./exports/sales_over_time.csv", index=False)
if 'seg_table' in globals():
    seg_table.to_csv("./exports/segments.csv", index=False)

print("Exports saved to ./exports (if tables existed).")